In [1]:
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read("../../config/config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

# Calculating accessibility statistics based on 2FSCA method
First, we create the table

## Step 1 table and calculations

In [15]:
SQL = """

DROP TABLE IF EXISTS step1_stats;

CREATE TABLE IF NOT EXISTS step1_stats (
    --- There is no need to store POI category in step1 statistics
    --- Any POI with the same originH3 will have the same step 1 value!
    --- If we were to store it, we can't easily reuse step1 values for adding new POIs of a new type in H3id
	--- The same applies for groupname - we only need to store TOTAL population per catchment at this stage
    --- We keep categorytype as some populations == people and some == households
	timeofday varchar(50),
	categorytype text,	
	catchmentid bigint,
    h3id character(15),
	ratio float
);

WITH step1 AS (
         SELECT 
                c3m.h3id,                
                catchments.timeofday,
                catchments.catchmentid,
                catchment_stats.categorytype,		 
				CASE 
            		WHEN SUM(catchment_stats.population) = 0
            		THEN 0 
            		ELSE 10000 / SUM(catchment_stats.population) 
            		END AS ratio
         FROM catchments
		 JOIN catchment_stats
         	ON catchment_stats.catchmentid = catchments.catchmentid                
         JOIN catchmenth3map c3m 
		 	ON catchment_stats.catchmentid = c3m.catchmentid
		 GROUP BY 
		 	c3m.h3id,                
            catchments.timeofday,
            catchments.catchmentid,
            catchment_stats.categorytype
     )
	 
	 INSERT INTO step1_stats 
	 (h3id,		
			timeofday,
			categorytype,			
			catchmentid,
			ratio
	 )
	 	SELECT 
            h3id,		
			timeofday,
			categorytype,			
			catchmentid,
			ratio
		 FROM step1;

CREATE INDEX IF NOT EXISTS step1_stats_agg_index ON public.step1_stats (
    catchmentid, categorytype, timeofday
);
CREATE INDEX IF NOT EXISTS step1_stats_cid ON public.step1_stats (catchmentid);
CREATE INDEX IF NOT EXISTS step1_stats_h3id ON public.step1_stats (h3id);

"""

execute_sql(SQL)